# Chapter 3.2 - Feature Calculation

In this JupyterNotebook, the code to calculate the parameters presented in Chapter 3.2., for both parameter calculation approaches and classification tasks as presented in Chapter 3.1., is provided. In total, as described in Chapter 3.1., four subsets of training data are calculated (PFAS – Profit, PFAS – Gain, AFPS – Profit, AFPS – Gain).

## Parametrization-First-Aggregation-Second Approach

#### Imports and Data Preprocessing

In [62]:
# Import Python modules
import pandas as pd
import numpy as np
import seaborn as sns
import os
from datetime import datetime
import time
from matplotlib import pyplot as plt

# Import Data
hh = pd.read_pickle('Data/households_cleaned_2016_simulation_normalized.pkl')

# Prepare the data subsets for seasonal and weekly differentiation criteria
workdays = [0,1,2,3,4]
weekend = [5,6]
spring = [4,5,10,11]
summer = [6,7,8,9]
winter = [12,1,2,3]

# Compute all data subsets using nested conditional indexing
hh_workday = hh[hh.index.get_level_values('Day').weekday.isin(workdays)]
hh_weekend = hh[hh.index.get_level_values('Day').weekday.isin(weekend)]
hh_spring = hh[hh.index.get_level_values('Day').month.isin(spring)]
hh_summer = hh[hh.index.get_level_values('Day').month.isin(summer)]
hh_winter = hh[hh.index.get_level_values('Day').month.isin(winter)]
hh_workday_spring = hh_workday[hh_workday.index.get_level_values('Day').month.isin(spring)] 
hh_workday_spring.name = '_spring/autumn_workday'
hh_weekend_spring = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(spring)] 
hh_weekend_spring.name = '_spring/autumn_weekend'
hh_workday_summer = hh_workday[hh_workday.index.get_level_values('Day').month.isin(summer)] 
hh_workday_summer.name = '_summer_workday'
hh_weekend_summer = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(summer)] 
hh_weekend_summer.name = '_summer_weekend'
hh_workday_winter = hh_workday[hh_workday.index.get_level_values('Day').month.isin(winter)] 
hh_workday_winter.name = '_winter_workday'
hh_weekend_winter = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(winter)] 
hh_weekend_winter.name = '_winter_weekend'

hh.head()

Household-IDs                   100061616572321  1000623286326231  \
Day        Time                                                     
2016-01-01 2016-01-01 00:30:00         0.000004          0.000051   
           2016-01-01 01:00:00         0.000013          0.000051   
           2016-01-01 01:30:00         0.000004          0.000048   
           2016-01-01 02:00:00         0.000011          0.000049   
           2016-01-01 02:30:00         0.000011          0.000055   

Household-IDs                   1000626063831495  1000634849535454  \
Day        Time                                                      
2016-01-01 2016-01-01 00:30:00          0.000041          0.000024   
           2016-01-01 01:00:00          0.000036          0.000039   
           2016-01-01 01:30:00          0.000033          0.000023   
           2016-01-01 02:00:00          0.000028          0.000023   
           2016-01-01 02:30:00          0.000025          0.000036   

Household-IDs                   1000643682431604  10005970974290684  \
Day        Time                                                       
2016-01-01 2016-01-01 00:30:00          0.000060           0.000033   
           2016-01-01 01:00:00          0.000052           0.000039   
           2016-01-01 01:30:00          0.000034           0.000041   
           2016-01-01 02:00:00          0.000028           0.000039   
           2016-01-01 02:30:00          0.000026           0.000038   

Household-IDs                   10005985465838684  10006012732591514  \
Day        Time                                                        
2016-01-01 2016-01-01 00:30:00           0.000058           0.000109   
           2016-01-01 01:00:00           0.000058           0.000109   
           2016-01-01 01:30:00           0.000058           0.000120   
           2016-01-01 02:00:00           0.000056           0.000120   
           2016-01-01 02:30:00           0.000058           0.000129   

Household-IDs                   10006013492777270  10006033110939880  ...  \
Day        Time                                                       ...   
2016-01-01 2016-01-01 00:30:00           0.000036           0.000038  ...   
           2016-01-01 01:00:00           0.000032           0.000038  ...   
           2016-01-01 01:30:00           0.000031           0.000040  ...   
           2016-01-01 02:00:00           0.000031           0.000037  ...   
           2016-01-01 02:30:00           0.000029           0.000039  ...   

Household-IDs                   1000655908257092480  1000655924023836928  \
Day        Time                                                            
2016-01-01 2016-01-01 00:30:00             0.000053             0.000054   
           2016-01-01 01:00:00             0.000051             0.000054   
           2016-01-01 01:30:00             0.000053             0.000054   
           2016-01-01 02:00:00             0.000050             0.000045   
           2016-01-01 02:30:00             0.000050             0.000046   

Household-IDs                   1000655926477497600  1000655952635175552  \
Day        Time                                                            
2016-01-01 2016-01-01 00:30:00             0.000067             0.000090   
           2016-01-01 01:00:00             0.000069             0.000105   
           2016-01-01 01:30:00             0.000059             0.000091   
           2016-01-01 02:00:00             0.000056             0.000043   
           2016-01-01 02:30:00             0.000061             0.000052   

Household-IDs                   1000655956248689536  1000655959333708544  \
Day        Time                                                            
2016-01-01 2016-01-01 00:30:00             0.000060             0.000013   
           2016-01-01 01:00:00             0.000055             0.000017   
           2016-01-01 01:30:00             0.000053             0.000012   
           2016-01-01 02:00:00      

#### Definition of Parameter Calculation Functions

In [63]:
###### PARAMETER CALCULATION ON DAILY LOAD PROFILES AND AGGREGATION WITH MEDIAN ######

###### Overall load shape ######

# Calculate the median ratio of mean daily consumption and maximum daily consumption
def daily_load_factor(hh):
    return (hh.groupby(level='Day').mean() / hh.groupby(level='Day').max()).median(axis='index')

# Calculate the median ratio of minimum daily consumption and median daily consumption
def daily_nonuniformity_coefficient(hh):
    return (hh.groupby(level='Day').min() / hh.groupby(level='Day').mean()).median(axis='index')

# Calculate the median ratio of minimum daily consumption and maximum daily consumption
def daily_range_factor(hh):
    return (hh.groupby(level='Day').min() / hh.groupby(level='Day').max()).median(axis='index')

###### Impact of certain daytimes ######

# Calculate the median of the impact of the night hours on the daily average
def night_impact(hh):
    return ((1/3)*(hh[hh.index.get_level_values('Time').hour.isin([23,0,1,2,3,4,5,6])].groupby(level='Day').mean() / hh.groupby(level='Day').mean())).median(axis='index')

# Calculate the median of the impact of the lunch hours on the daily average
def lunch_impact(hh):
    return ((1/8)*(hh[hh.index.get_level_values('Time').hour.isin([11,12,13])].groupby(level='Day').mean() / hh.groupby(level='Day').mean()).median(axis='index'))

# Calculate the median of the impact of the end-of-work hours on the daily average
def end_of_work_impact(hh):
    return ((1/6)*(hh[hh.index.get_level_values('Time').hour.isin([16,17,18,19])].groupby(level='Day').mean() / hh.groupby(level='Day').mean())).median(axis='index')

###### Slopes ######

# Calculate the median difference between the consumption at 10am and 6am 
def morning_slope(hh):
    return hh.groupby(level='Day').agg(lambda s : s[20]-s[12]).median(axis='index')

# Calculate the median difference between the consumption at 11pm and 9pm 
def night_slope(hh):
    return hh.groupby(level='Day').agg(lambda s : s[46]-s[42]).median(axis='index')

###### Peaks ######

# Calculate the Median of the daily demand peaks
def median_daily_maximum_demand(hh):
    return hh.groupby(level='Day').max().median(axis='index')

# Calculate the mode of the daily time of maximum use
def maximum_tou(hh):
    # Calculate the argmax (=timestamp) for each day, format the timestamp, and calculate the mode for each household 
    return (hh.groupby(level='Day')
               .idxmax() 
               .apply(np.vectorize(lambda x : x[1].time()))
               .mode(axis='index').iloc[0]
               .map(np.vectorize(lambda tou : int(str(tou)[:2])*2 + 1 if (str(tou)[3:5]=='30') else int(str(tou)[:2])*2)))

# Calculate the Median of the daily demand minimums
def median_daily_minimum_demand(hh):
    return hh.groupby(level='Day').min().median(axis='index')

# Calculate the mode of the daily time of minimum use
def minimum_tou(hh):
    # Calculate the argmax (=timestamp) for each day, format the timestamp, and calculate the mode for each household 
    return (hh.groupby(level='Day')
               .idxmin() 
               .apply(np.vectorize(lambda x : x[1].time()))
               .mode(axis='index').iloc[0]
               .map(np.vectorize(lambda tou : int(str(tou)[:2])*2 + 1 if (str(tou)[3:5]=='30') else int(str(tou)[:2])*2)))

##### Frequency-based #####

# Calculate the maximum of the absolute values of the FFT-Coefficients
def fft_peak(hh):
    # Old: return hh.apply(lambda series : abs(np.fft.rfft(series.to_numpy())).max(), axis='index')
    return hh.groupby(level='Day').agg(lambda series : abs(np.fft.rfft(series.to_numpy())).max()).median(axis='index')

##### Central Statistical Moments ######

# Calculate the median of the standard deviation of the daily load profiles
def variance(hh):
    return hh.var(level='Day').median(axis='index')

# Calculate the median of the skewness of the daily load profiles
def skewness(hh):
    return hh.skew(level='Day').median(axis='index')

# Calculate the median of the kurtosis of the daily load profiles
def kurtosis(hh):
    return hh.kurtosis(level='Day').median(axis='index')

##### Similarity to PV Systems ######

def pv_correlation(hh):
    pv = pd.read_pickle('PV_full.pkl')
    data = hh.copy()
    data.index = data.index.droplevel('Day')
    return data.apply(lambda series : series.corr(pv['Power']), axis='index')

###### Weekly Differences ###### 

# Calculate the ratio of the daily average workday consumption and the daily average weekend consumption
def workday_weekend_ratio(hh_workday, hh_weekend):
    return hh_workday.groupby(level='Day').sum().median() / hh_weekend.groupby(level='Day').sum().median()

###### Yearly Trend ######

# Calculate the ratio of the consumption in summer months to the consumption in winter months
def summer_winter_ratio(hh_summer, hh_winter):
    return hh_summer.sum(axis='rows') / hh_winter.sum(axis='rows')

#### Parameter Value Calculation

In [65]:
# Collect all parameters in a list to enable iteration
parameter_functions = [daily_load_factor, daily_nonuniformity_coefficient, daily_range_factor, night_impact, lunch_impact, end_of_work_impact, morning_slope, night_slope, median_daily_maximum_demand, maximum_tou, median_daily_minimum_demand, minimum_tou, fft_peak, variance, skewness, kurtosis, pv_correlation, workday_weekend_ratio, summer_winter_ratio]  

# Collect the data subsets in a list to enable iteration
data_subsets = [hh_workday_spring, hh_weekend_spring, hh_workday_summer, hh_weekend_summer, hh_workday_winter, hh_weekend_winter] 

# Calculate all parameters for each household and save them in a DataFrame
parameter_values = pd.DataFrame()
parameter_values.index.name = 'Household-ID'
parameter_values.columns.name = 'Parameter'
for parameter in parameter_functions:
    for subset in data_subsets:
        # Use the special data subsets for the parameters on year-level:
        if parameter.__name__ == 'workday_weekend_ratio':
            parameter_values['workday_weekend_ratio_spring/autumn'] = workday_weekend_ratio(hh_workday_spring, hh_weekend_spring)
            parameter_values['workday_weekend_ratio_summer'] = workday_weekend_ratio(hh_workday_summer, hh_weekend_summer)
            parameter_values['workday_weekend_ratio_winter'] = workday_weekend_ratio(hh_workday_winter, hh_weekend_winter)
        elif parameter.__name__ == 'summer_winter_ratio':
            parameter_values['summer_winter_ratio_workday'] = summer_winter_ratio(hh_workday_summer, hh_workday_winter)
            parameter_values['summer_winter_ratio_weekend'] = summer_winter_ratio(hh_weekend_summer, hh_weekend_winter)
        # Use the other data subsets for the parameter on day-level:
        else:
            parameter_values[parameter.__name__ + subset.name] = parameter(subset)

parameter_values.to_pickle('Data/Parameter_Values_PFAS_Non-Aggregated_{}.pkl'.format(str(datetime.fromtimestamp(time.time()).date())))    

parameter_values

Parameter,daily_load_factor_spring/autumn_workday,daily_load_factor_spring/autumn_weekend,daily_load_factor_summer_workday,daily_load_factor_summer_weekend,daily_load_factor_winter_workday,daily_load_factor_winter_weekend,daily_nonuniformity_coefficient_spring/autumn_workday,daily_nonuniformity_coefficient_spring/autumn_weekend,daily_nonuniformity_coefficient_summer_workday,daily_nonuniformity_coefficient_summer_weekend,...,pv_correlation_spring/autumn_weekend,pv_correlation_summer_workday,pv_correlation_summer_weekend,pv_correlation_winter_workday,pv_correlation_winter_weekend,workday_weekend_ratio_spring/autumn,workday_weekend_ratio_summer,workday_weekend_ratio_winter,summer_winter_ratio_workday,summer_winter_ratio_weekend
Household-IDs,,,,,,,,,,,,,,,,,,,,,
100061616572321,0.213354,0.311087,0.558066,0.514509,0.215338,0.350949,0.195332,0.194164,0.322504,0.218593,...,0.029166,-0.070358,0.043842,0.030139,0.078188,0.835530,1.185439,0.935116,4.754739,4.009599
1000623286326231,0.428551,0.423693,0.409185,0.380454,0.505519,0.476940,0.453365,0.440515,0.110516,0.129367,...,0.082865,0.287194,0.313186,-0.129226,-0.033074,0.824004,1.290621,0.933809,3.792021,3.038055
1000626063831495,0.254768,0.294691,0.343003,0.311232,0.337878,0.316153,0.149206,0.209602,0.135577,0.136522,...,0.059304,0.297350,0.291069,-0.163828,-0.099652,0.853271,1.047831,0.892595,0.895504,0.890229
1000634849535454,0.503351,0.480552,0.400725,0.484675,0.445535,0.462211,0.518560,0.516201,0.548475,0.610236,...,0.070545,0.021550,0.059276,-0.156275,-0.087399,0.903726,0.989770,0.903824,1.521461,1.357305
1000643682431604,0.495062,0.443975,0.517778,0.495597,0.498647,0.484927,0.380606,0.409627,0.273825,0.330104,...,-0.008328,-0.141040,-0.220975,-0.087900,-0.075350,0.992371,1.839328,1.003181,2.239565,1.771086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000655959333708544,0.405142,0.378980,0.632014,0.600853,0.407179,0.394337,0.366154,0.354271,0.606223,0.595656,...,0.027800,0.017951,0.038269,-0.029713,-0.012960,1.004568,1.095393,1.062700,4.198671,3.990208
1000655960699468928,0.328895,0.328725,0.481127,0.502588,0.435074,0.321820,0.192136,0.215328,0.229059,0.226650,...,-0.148673,-0.258889,-0.203741,-0.272640,-0.205245,0.891196,1.074251,0.954293,0.493994,0.375758
1000656005414328704,0.686918,0.667921,0.730524,0.709929,0.717708,0.684896,0.791501,0.813203,0.820745,0.832177,...,-0.107439,0.004871,-0.076466,-0.157429,-0.116850,1.094314,0.993781,1.070078,1.022868,0.975696


#### Aggregation on Community Level (Community Profit)

In [66]:
import pandas as pd

# Import the simulation data to access the household-IDs and target variable
simulation_data = pd.read_pickle('Data/Simulation_Results.pkl')

# Import the non-aggregated parameter values calculated in the last step
parameter_values = pd.read_pickle('Data/Parameter_Values_PFAS_Non-Aggregated_2020-09-25')

# Create a DataFrame to store the aggregated parameter values
parameter_values_agg = pd.DataFrame(index=simulation_data.index, columns=parameter_values.columns)

# For all households within each community, sum up the parameter values:
for community in simulation_data.index:
    for parameter in parameter_values.columns:
        parameter_values_agg.loc[community, parameter] = (parameter_values.loc[simulation_data.loc[community,'Battery/PV_1'], parameter] +
                                                           parameter_values.loc[simulation_data.loc[community,'Battery/PV_2'], parameter] +
                                                           parameter_values.loc[simulation_data.loc[community,'PV'], parameter] +
                                                           parameter_values.loc[simulation_data.loc[community,'Consumer_1'], parameter] +
                                                           parameter_values.loc[simulation_data.loc[community,'Consumer_2'], parameter])

# Add the corresponding Community Profit of each community, to obtain the final training data with features (parameters) and target (Community Profit)        
parameter_values_agg['CommunityProfit_Class'] = simulation_data['CommunityProfit_Class']

# Save the final training data
parameter_values_agg.to_pickle('Data/Parameter_Values_PFAS_Profit_{}.pkl'.format(str(datetime.fromtimestamp(time.time()).date())))    

parameter_values_agg

Parameter,daily_load_factor_spring/autumn_workday,daily_load_factor_spring/autumn_weekend,daily_load_factor_summer_workday,daily_load_factor_summer_weekend,daily_load_factor_winter_workday,daily_load_factor_winter_weekend,daily_nonuniformity_coefficient_spring/autumn_workday,daily_nonuniformity_coefficient_spring/autumn_weekend,daily_nonuniformity_coefficient_summer_workday,daily_nonuniformity_coefficient_summer_weekend,...,pv_correlation_summer_workday,pv_correlation_summer_weekend,pv_correlation_winter_workday,pv_correlation_winter_weekend,workday_weekend_ratio_spring/autumn,workday_weekend_ratio_summer,workday_weekend_ratio_winter,summer_winter_ratio_workday,summer_winter_ratio_weekend,CommunityProfit_Class
1,2.63347,2.64708,2.82121,2.75585,2.59359,2.61659,2.63906,2.87985,2.18801,2.25184,...,1.13224,1.45548,0.0722735,0.309436,4.49859,5.43861,4.76581,11.1854,9.29603,Medium
2,2.00872,1.98917,2.05474,1.80822,2.22968,2.02285,1.83828,1.85434,0.880315,0.937656,...,1.26768,1.20115,-0.0557157,-0.0688357,5.27135,6.01493,5.10674,12.2804,11.0984,High
3,2.02736,2.00322,2.25933,2.17689,2.18666,2.19428,1.61512,1.6538,1.43546,1.45402,...,-0.117007,0.219427,-0.370859,-0.16649,4.99741,5.20427,5.06936,7.51563,7.23033,High
4,1.90105,1.868,1.7685,1.84588,2.07066,2.10159,1.62489,1.57892,1.68565,1.69193,...,0.478157,0.45663,-0.386586,-0.0157196,4.74234,5.37937,5.13336,5.35244,4.95736,Low
5,1.85376,1.93085,2.30072,2.08569,2.02995,2.02266,1.68429,1.62028,1.56805,1.53965,...,-0.193926,0.283975,-0.502685,-0.304117,4.77229,5.40144,4.65855,7.91869,7.29004,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,1.4245,1.56677,1.28082,1.40098,1.49503,1.68675,1.57984,1.58085,0.904102,1.15565,...,0.0459276,0.459332,-0.365106,-0.174105,4.97853,6.15059,4.77092,6.59265,5.8227,Medium
997,1.56963,1.67231,1.78055,1.78481,1.96622,1.79128,1.82161,1.78185,1.51863,1.40667,...,-0.477714,0.119267,-0.178213,-0.177762,4.77942,5.57009,4.77355,6.58852,5.9069,Medium
998,1.73857,1.66839,1.85283,1.82804,1.77242,1.78489,1.35114,1.32751,1.10261,1.13766,...,-0.61251,-0.194493,-0.559599,0.0557018,4.18156,4.54969,4.32774,9.88905,9.79426,Low
999,2.1085,2.0429,2.2534,2.10848,2.37329,2.23464,1.8547,1.9783,1.64417,1.777,...,-0.0554478,-0.0112208,-1.06668,-0.412867,4.41608,4.60331,4.34276,8.51203,7.37906,Low


#### Aggregation on Community Level (Community Gain)

In [67]:
import pandas as pd

# Import the simulation data to access the household-IDs and target variable
simulation_data = pd.read_pickle('Data/Simulation_Results.pkl')

# Import the non-aggregated parameter values calculated in the last step
parameter_values = pd.read_pickle('Data/Parameter_Values_PFAS_Non-Aggregated_2020-09-25')

# Create a DataFrame to store the aggregated parameter values
parameter_values_agg = pd.DataFrame(index=simulation_data.index, columns=parameter_values.columns)

# For both consumer households within each community, sum up the parameter values:
for community in simulation_data.index:
    for parameter in parameter_values.columns:
        parameter_values_agg.loc[community, parameter] = (parameter_values.loc[simulation_data.loc[community,'Consumer_1'], parameter] +
                                                          parameter_values.loc[simulation_data.loc[community,'Consumer_2'], parameter])

# Add the corresponding Community Gain of each community, to obtain the final training data with features (parameters) and target (Community Gain)
parameter_values_agg['CommunityGain_Class'] = simulation_data['CommunityGain_Class']

# Save the final training data
parameter_values_agg.to_pickle('Data/Parameter_Values_PFAS_Gain_{}.pkl'.format(str(datetime.fromtimestamp(time.time()).date())))    

parameter_values_agg

Parameter,daily_load_factor_spring/autumn_workday,daily_load_factor_spring/autumn_weekend,daily_load_factor_summer_workday,daily_load_factor_summer_weekend,daily_load_factor_winter_workday,daily_load_factor_winter_weekend,daily_nonuniformity_coefficient_spring/autumn_workday,daily_nonuniformity_coefficient_spring/autumn_weekend,daily_nonuniformity_coefficient_summer_workday,daily_nonuniformity_coefficient_summer_weekend,...,pv_correlation_summer_workday,pv_correlation_summer_weekend,pv_correlation_winter_workday,pv_correlation_winter_weekend,workday_weekend_ratio_spring/autumn,workday_weekend_ratio_summer,workday_weekend_ratio_winter,summer_winter_ratio_workday,summer_winter_ratio_weekend,CommunityGain_Class
1,0.770333,0.759663,0.79414,0.767272,0.665286,0.613473,0.582344,0.819026,0.417604,0.476227,...,0.436843,0.634665,0.00976121,0.272508,1.91947,2.36492,1.76137,6.98251,5.24905,Medium
2,0.792863,0.812554,0.91169,0.797375,0.864885,0.682151,0.611366,0.621127,0.43467,0.470582,...,0.165527,0.165321,0.0133505,0.136047,2.15418,2.60938,2.06557,4.4405,3.8036,High
3,0.738785,0.770617,0.805736,0.741452,0.755621,0.787908,0.694143,0.690135,0.621032,0.606263,...,-0.106641,0.0644488,-0.255352,0.0484907,1.84669,2.06392,2.20726,1.74465,1.79729,Medium
4,0.6103,0.639771,0.567914,0.669573,0.637614,0.662514,0.797903,0.784695,0.952477,0.975523,...,-0.210974,0.0170494,-0.326788,-0.105485,1.84759,2.2789,2.03348,2.13409,1.8467,Medium
5,0.725147,0.640184,0.874344,0.684253,0.705606,0.667796,0.565531,0.487895,0.439847,0.434919,...,-0.195158,-0.0449871,-0.10688,-0.0984599,1.92502,2.26269,1.89549,3.15313,2.85467,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.702804,0.721426,0.503653,0.543408,0.71672,0.734607,0.814991,0.814611,0.415541,0.620578,...,-0.133307,-0.0635992,-0.182661,-0.188038,2.04057,2.93985,2.05924,2.2884,1.7563,Low
997,0.621774,0.663229,0.625149,0.641883,0.715959,0.621976,0.516381,0.591642,0.52392,0.577805,...,-0.115723,0.103883,-0.18782,-0.0524897,1.93736,2.1118,1.89605,2.27999,1.94101,High
998,0.826011,0.788444,0.872218,0.857808,0.8623,0.825111,0.542655,0.510837,0.391417,0.433553,...,-0.230908,-0.290016,-0.30292,-0.10699,1.74957,1.85007,1.84205,4.99866,4.96712,Low
999,0.856969,0.909359,0.871723,0.847192,0.937319,0.949932,0.690755,0.740588,0.78825,0.836121,...,-0.0264571,-0.244841,-0.523753,-0.316158,2.07021,2.04333,1.95936,2.49044,2.18496,Low


## Aggregation-First-Parametrization-Second Approach

#### Definition of Parameter Calculation Functions

In [68]:
###### PARAMETER CALCULATION ON AVERAGE DAILY LOAD PROFILES ######

###### Overall load shape ######

# Calculate the median ratio of mean daily consumption and maximum daily consumption
def daily_load_factor(hh):
    return hh.mean(axis='index') / hh.max(axis='index')

# Calculate the median ratio of minimum daily consumption and median daily consumption
def daily_nonuniformity_coefficient(hh):
    return hh.min(axis='index') / hh.mean(axis='index')

# Calculate the median ratio of minimum daily consumption and maximum daily consumption
def daily_range_factor(hh):
    return hh.min(axis='index') / hh.max(axis='index')

###### Impact of certain daytimes ######

# Calculate the median of the impact of the night hours on the daily average
def night_impact(hh):
    return (1/3)*(hh.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,12,46,47],:].mean(axis='index') / hh.mean(axis='index'))

# Calculate the median of the impact of the lunch hours on the daily average
def lunch_impact(hh):
    return (1/8)*(hh.iloc[[22,23,24,25,26],:].mean(axis='index') / hh.mean(axis='index'))

# Calculate the median of the impact of the end-of-work hours on the daily average
def end_of_work_impact(hh):
    return (1/6)*(hh.iloc[[32,33,34,35,36,37,38],:].mean(axis='index') / hh.mean(axis='index'))

###### Slopes ######

# Calculate the median difference between the consumption at 10am and 6am 
def morning_slope(hh):
    return hh.iloc[20,:] - hh.iloc[12,:]

# Calculate the median difference between the consumption at 11pm and 9pm 
def night_slope(hh):
    return hh.iloc[46,:] - hh.iloc[42,:]

###### Peaks ######

# Calculate the Median of the daily demand peaks
def median_daily_maximum_demand(hh):
    return hh.max()

# Calculate the mode of the daily time of maximum use
def maximum_tou(hh):
    # Calculate the argmax (=timestamp) for each day, format the timestamp, and calculate the mode for each household 
    return hh.idxmax().map(np.vectorize(lambda tou : int(str(tou)[:2])*2 + 1 if (str(tou)[3:5]=='30') else int(str(tou)[:2])*2))

# Calculate the Median of the daily demand peaks
def median_daily_minimum_demand(hh):
    return hh.min()

# Calculate the mode of the daily time of maximum use
def minimum_tou(hh):
    # Calculate the argmax (=timestamp) for each day, format the timestamp, and calculate the mode for each household 
    return hh.idxmin().map(np.vectorize(lambda tou : int(str(tou)[:2])*2 + 1 if (str(tou)[3:5]=='30') else int(str(tou)[:2])*2))

##### Frequency-based #####

# Calculate the maximum of the absolute values of the FFT-Coefficients
def fft_peak(hh):
    return hh.apply(lambda series : abs(np.fft.rfft(series.to_numpy())).max())

##### Central Statistical Moments ######

# Calculate the median of the standard deviation of the daily load profiles
def variance(hh):
    return hh.var()

# Calculate the median of the skewness of the daily load profiles
def skewness(hh):
    return hh.skew()

# Calculate the median of the kurtosis of the daily load profiles
def kurtosis(hh):
    return hh.kurtosis()

def pv_correlation(hh):
    pv = pd.read_pickle('PV_mean.pkl')
    return hh.apply(lambda series : series.corr(pv['Power']), axis='index')

###### Weekly Differences ###### 

# Calculate the ratio of the daily average workday consumption and the daily average weekend consumption
def workday_weekend_ratio(hh_workday, hh_weekend):
    return hh_workday.sum(axis='rows') / hh_weekend.sum(axis='rows')

###### Yearly Trend ######

# Calculate the ratio of the consumption in summer months to the consumption in winter months
def summer_winter_ratio(hh_summer, hh_winter):
    return hh_summer.sum(axis='rows') / hh_winter.sum(axis='rows')

#### Parameter Value Calculation (Community Profit)

In [69]:
import pandas as pd 
import numpy as np

# Utility function to sum up the load profiles of each community 
def aggregate_community_load_profiles(hh_av):   
    result = pd.DataFrame(index=hh_av.index, columns=simulation.index)
    for community in simulation.index:
        households = simulation.iloc[community-1,:5].to_list()
        result[community] = hh_av[households].sum(axis='columns')
    return result

hh = pd.read_pickle('Data/households_cleaned_2016_simulation_normalized.pkl')
simulation = pd.read_pickle('Data/Simulation_Results.pkl')

# Prepare the data subsets for seasonal and weekly differentiation criteria
workdays = [0,1,2,3,4]
weekend = [5,6]
spring = [4,5,10,11]
summer = [6,7,8,9]
winter = [12,1,2,3]

# Compute all data subsets using nested conditional indexing
hh_workday = hh[hh.index.get_level_values('Day').weekday.isin(workdays)]
hh_weekend = hh[hh.index.get_level_values('Day').weekday.isin(weekend)]
hh_spring = hh[hh.index.get_level_values('Day').month.isin(spring)]
hh_summer = hh[hh.index.get_level_values('Day').month.isin(summer)]
hh_winter = hh[hh.index.get_level_values('Day').month.isin(winter)]
hh_workday_spring = hh_workday[hh_workday.index.get_level_values('Day').month.isin(spring)] 
hh_weekend_spring = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(spring)] 
hh_workday_summer = hh_workday[hh_workday.index.get_level_values('Day').month.isin(summer)] 
hh_weekend_summer = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(summer)] 
hh_workday_winter = hh_workday[hh_workday.index.get_level_values('Day').month.isin(winter)] 
hh_weekend_winter = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(winter)] 

# Calculte average load profiles for all subsets
hh_workday_spring_av = hh_workday_spring.groupby(by=hh_workday_spring.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_weekend_spring_av = hh_weekend_spring.groupby(by=hh_weekend_spring.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_workday_summer_av = hh_workday_summer.groupby(by=hh_workday_summer.index.droplevel('Day').map(lambda idx : idx.time())).median() 
hh_weekend_summer_av = hh_weekend_summer.groupby(by=hh_weekend_summer.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_workday_winter_av = hh_workday_winter.groupby(by=hh_workday_winter.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_weekend_winter_av = hh_weekend_winter.groupby(by=hh_weekend_winter.index.droplevel('Day').map(lambda idx : idx.time())).median()

# Sum up the average load profiles of the communities for each subset 
communities_workday_spring_sum = aggregate_community_load_profiles(hh_workday_spring_av)
communities_workday_spring_sum.name = '_spring/autumn_workday'
communities_weekend_spring_sum = aggregate_community_load_profiles(hh_weekend_spring_av)
communities_weekend_spring_sum.name = '_spring/autumn_weekend'
communities_workday_summer_sum = aggregate_community_load_profiles(hh_workday_summer_av)
communities_workday_summer_sum.name = '_summer_workday'
communities_weekend_summer_sum = aggregate_community_load_profiles(hh_weekend_summer_av)
communities_weekend_summer_sum.name = '_summer_weekend'
communities_workday_winter_sum = aggregate_community_load_profiles(hh_workday_winter_av)
communities_workday_winter_sum.name = '_winter_workday'
communities_weekend_winter_sum = aggregate_community_load_profiles(hh_weekend_winter_av)
communities_weekend_winter_sum.name = '_winter_weekend'

print('Done')

Done


In [70]:
from datetime import datetime
import time

# Import the simulation data to access the household-IDs and target variable
simulation = pd.read_pickle('Data/Simulation_Results.pkl')

# Collect all parameters in a list to enable iteration
parameter_functions = [daily_load_factor, daily_nonuniformity_coefficient, daily_range_factor, night_impact, lunch_impact, end_of_work_impact, morning_slope, night_slope, median_daily_maximum_demand, maximum_tou, median_daily_minimum_demand, minimum_tou, fft_peak, variance, skewness, kurtosis, pv_correlation]  

# Collect the data subsets in a list to enable iteration
data_subsets = [communities_workday_spring_sum, communities_weekend_spring_sum, communities_workday_summer_sum, communities_weekend_summer_sum, communities_workday_winter_sum, communities_weekend_winter_sum]

# Calculate all parameters for each household and save them in a DataFrame
parameter_values = pd.DataFrame()
parameter_values.index.name = 'Household-ID'
parameter_values.columns.name = 'Parameter'
for parameter in parameter_functions:
    for subset in data_subsets:
        parameter_values[parameter.__name__ + subset.name] = parameter(subset)

# Add the parameters on year-level separately since they require different subsets
parameter_values['workday_weekend_ratio_spring/autumn'] = workday_weekend_ratio(communities_workday_spring_sum, communities_weekend_spring_sum)
parameter_values['workday_weekend_ratio_summer'] = workday_weekend_ratio(communities_workday_summer_sum, communities_weekend_summer_sum)
parameter_values['workday_weekend_ratio_winter'] = workday_weekend_ratio(communities_workday_winter_sum, communities_weekend_winter_sum)
parameter_values['summer_winter_ratio_workday'] = summer_winter_ratio(communities_workday_summer_sum, communities_workday_winter_sum)
parameter_values['summer_winter_ratio_weekend'] = summer_winter_ratio(communities_weekend_summer_sum, communities_weekend_winter_sum)

# Add the corresponding Community Profit of each community, to obtain the final training data with features (parameters) and target (Community Profit)
parameter_values['CommunityProfit_Class'] = simulation['CommunityProfit_Class']

# Save the final training data
parameter_values.to_pickle('Data/Parameter_Values_AFPS_Profit_{}.pkl'.format(str(datetime.fromtimestamp(time.time()).date())))     

parameter_values

Parameter,daily_load_factor_spring/autumn_workday,daily_load_factor_spring/autumn_weekend,daily_load_factor_summer_workday,daily_load_factor_summer_weekend,daily_load_factor_winter_workday,daily_load_factor_winter_weekend,daily_nonuniformity_coefficient_spring/autumn_workday,daily_nonuniformity_coefficient_spring/autumn_weekend,daily_nonuniformity_coefficient_summer_workday,daily_nonuniformity_coefficient_summer_weekend,...,pv_correlation_summer_workday,pv_correlation_summer_weekend,pv_correlation_winter_workday,pv_correlation_winter_weekend,workday_weekend_ratio_spring/autumn,workday_weekend_ratio_summer,workday_weekend_ratio_winter,summer_winter_ratio_workday,summer_winter_ratio_weekend,CommunityProfit_Class
1,0.821693,0.787444,0.627893,0.683353,0.822387,0.871099,0.811113,0.725711,0.516604,0.536978,...,0.640161,0.753639,0.046456,0.547647,0.925478,1.059751,0.981495,1.625245,1.505230,Medium
2,0.674074,0.712161,0.462176,0.480470,0.722233,0.747166,0.696467,0.700924,0.239397,0.292638,...,0.461864,0.514300,0.068251,0.074777,1.004006,1.228799,1.028415,2.462522,2.060952,High
3,0.689060,0.673424,0.700077,0.686754,0.669222,0.755270,0.664066,0.643800,0.749171,0.652043,...,-0.661606,-0.104189,-0.345373,0.049928,0.988679,1.029090,0.989551,1.324356,1.273473,High
4,0.525937,0.556392,0.618357,0.741031,0.559138,0.610578,0.642972,0.640037,0.628755,0.610256,...,0.166840,0.365292,-0.209289,-0.005252,0.961352,1.066410,1.027067,0.936090,0.901555,Low
5,0.623960,0.566864,0.662351,0.703304,0.692451,0.618912,0.636152,0.576227,0.695702,0.713829,...,-0.024315,0.129749,-0.529679,-0.094743,0.933731,1.055775,0.910930,1.670055,1.440936,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.598693,0.648489,0.551401,0.567370,0.482324,0.642999,0.725854,0.752072,0.469315,0.448931,...,-0.001457,0.359505,-0.295615,0.250016,0.938514,1.135903,0.953950,1.113513,0.935146,Medium
997,0.622067,0.735081,0.613495,0.624024,0.788024,0.668965,0.791628,0.768978,0.713251,0.669140,...,-0.635781,-0.336060,-0.128776,0.373972,0.958678,1.074269,1.134097,0.720825,0.760969,Medium
998,0.634718,0.654231,0.580867,0.651711,0.687974,0.749169,0.731494,0.688584,0.601265,0.541557,...,-0.537694,-0.469525,-0.569804,-0.005011,0.870835,0.880555,0.875127,1.661532,1.651288,Low
999,0.623378,0.709127,0.712325,0.717040,0.665083,0.703362,0.685658,0.706732,0.640863,0.622938,...,-0.104024,0.035149,-0.440252,-0.079201,0.907467,0.969519,0.907465,1.352629,1.266054,Low


#### Parameter Value Calculation (Community Gain)

In [71]:
import pandas as pd 
import numpy as np

# Utility function to sum up the load profiles of each community 
def aggregate_community_load_profiles(hh_av):   
    result = pd.DataFrame(index=hh_av.index, columns=simulation.index)
    for community in simulation.index:
        households = simulation.iloc[community-1,3:5].to_list()
        result[community] = hh_av[households].sum(axis='columns')
    return result

hh = pd.read_pickle('Data/households_cleaned_2016_simulation_normalized.pkl')
simulation = pd.read_pickle('Data/Simulation_Results.pkl')

workdays = [0,1,2,3,4]
weekend = [5,6]
spring = [4,5,10,11]
summer = [6,7,8,9]
winter = [12,1,2,3]

# Prepare the data subsets for seasonal and weekly differentiation criteria
hh_workday = hh[hh.index.get_level_values('Day').weekday.isin(workdays)]
hh_weekend = hh[hh.index.get_level_values('Day').weekday.isin(weekend)]
hh_spring = hh[hh.index.get_level_values('Day').month.isin(spring)]
hh_summer = hh[hh.index.get_level_values('Day').month.isin(summer)]
hh_winter = hh[hh.index.get_level_values('Day').month.isin(winter)]
hh_workday_spring = hh_workday[hh_workday.index.get_level_values('Day').month.isin(spring)] 
hh_weekend_spring = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(spring)] 
hh_workday_summer = hh_workday[hh_workday.index.get_level_values('Day').month.isin(summer)] 
hh_weekend_summer = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(summer)] 
hh_workday_winter = hh_workday[hh_workday.index.get_level_values('Day').month.isin(winter)] 
hh_weekend_winter = hh_weekend[hh_weekend.index.get_level_values('Day').month.isin(winter)] 

# Calculte average load profiles for all subsets
hh_workday_spring_av = hh_workday_spring.groupby(by=hh_workday_spring.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_weekend_spring_av = hh_weekend_spring.groupby(by=hh_weekend_spring.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_workday_summer_av = hh_workday_summer.groupby(by=hh_workday_summer.index.droplevel('Day').map(lambda idx : idx.time())).median() 
hh_weekend_summer_av = hh_weekend_summer.groupby(by=hh_weekend_summer.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_workday_winter_av = hh_workday_winter.groupby(by=hh_workday_winter.index.droplevel('Day').map(lambda idx : idx.time())).median()
hh_weekend_winter_av = hh_weekend_winter.groupby(by=hh_weekend_winter.index.droplevel('Day').map(lambda idx : idx.time())).median()

# Sum up the average load profiles of the communities for each subset 
communities_workday_spring_sum = aggregate_community_load_profiles(hh_workday_spring_av)
communities_workday_spring_sum.name = '_spring/autumn_workday'
communities_weekend_spring_sum = aggregate_community_load_profiles(hh_weekend_spring_av)
communities_weekend_spring_sum.name = '_spring/autumn_weekend'
communities_workday_summer_sum = aggregate_community_load_profiles(hh_workday_summer_av)
communities_workday_summer_sum.name = '_summer_workday'
communities_weekend_summer_sum = aggregate_community_load_profiles(hh_weekend_summer_av)
communities_weekend_summer_sum.name = '_summer_weekend'
communities_workday_winter_sum = aggregate_community_load_profiles(hh_workday_winter_av)
communities_workday_winter_sum.name = '_winter_workday'
communities_weekend_winter_sum = aggregate_community_load_profiles(hh_weekend_winter_av)
communities_weekend_winter_sum.name = '_winter_weekend'

print('Done')

Done


In [72]:
from datetime import datetime
import time

# Import the simulation data to access the household-IDs and target variable
simulation = pd.read_pickle('Data/Simulation_Results.pkl')

# Collect all parameters in a list to enable iteration
parameter_functions = [daily_load_factor, daily_nonuniformity_coefficient, daily_range_factor, night_impact, lunch_impact, end_of_work_impact, morning_slope, night_slope, median_daily_maximum_demand, maximum_tou, median_daily_minimum_demand, minimum_tou, fft_peak, variance, skewness, kurtosis, pv_correlation]  

# Collect the data subsets in a list to enable iteration
data_subsets = [communities_workday_spring_sum, communities_weekend_spring_sum, communities_workday_summer_sum, communities_weekend_summer_sum, communities_workday_winter_sum, communities_weekend_winter_sum]

# Calculate all parameters for each household and save them in a DataFrame
parameter_values = pd.DataFrame()
parameter_values.index.name = 'Household-ID'
parameter_values.columns.name = 'Parameter'
for parameter in parameter_functions:
    for subset in data_subsets:
        parameter_values[parameter.__name__ + subset.name] = parameter(subset)
        
# Add the parameters on year-level separately since they require different subsets
parameter_values['workday_weekend_ratio_spring/autumn'] = workday_weekend_ratio(communities_workday_spring_sum, communities_weekend_spring_sum)
parameter_values['workday_weekend_ratio_summer'] = workday_weekend_ratio(communities_workday_summer_sum, communities_weekend_summer_sum)
parameter_values['workday_weekend_ratio_winter'] = workday_weekend_ratio(communities_workday_winter_sum, communities_weekend_winter_sum)
parameter_values['summer_winter_ratio_workday'] = summer_winter_ratio(communities_workday_summer_sum, communities_workday_winter_sum)
parameter_values['summer_winter_ratio_weekend'] = summer_winter_ratio(communities_weekend_summer_sum, communities_weekend_winter_sum)

# Add the corresponding Community Gain of each community, to obtain the final training data with features (parameters) and target (Community Gain)
parameter_values['CommunityGain_Class'] = simulation['CommunityGain_Class']

# Save the final training data
parameter_values.to_pickle('Data/Parameter_Values_AFPS_Gain_{}.pkl'.format(str(datetime.fromtimestamp(time.time()).date())))     

parameter_values

Parameter,daily_load_factor_spring/autumn_workday,daily_load_factor_spring/autumn_weekend,daily_load_factor_summer_workday,daily_load_factor_summer_weekend,daily_load_factor_winter_workday,daily_load_factor_winter_weekend,daily_nonuniformity_coefficient_spring/autumn_workday,daily_nonuniformity_coefficient_spring/autumn_weekend,daily_nonuniformity_coefficient_summer_workday,daily_nonuniformity_coefficient_summer_weekend,...,pv_correlation_summer_workday,pv_correlation_summer_weekend,pv_correlation_winter_workday,pv_correlation_winter_weekend,workday_weekend_ratio_spring/autumn,workday_weekend_ratio_summer,workday_weekend_ratio_winter,summer_winter_ratio_workday,summer_winter_ratio_weekend,CommunityGain_Class
1,0.622909,0.713187,0.520280,0.643711,0.646312,0.749715,0.683838,0.544220,0.318495,0.347838,...,0.505441,0.689129,-0.077331,0.589335,1.052049,1.125014,0.943167,3.163535,2.652183,Medium
2,0.668288,0.684871,0.517426,0.491560,0.670343,0.619147,0.603318,0.630520,0.279976,0.424259,...,-0.261801,-0.250586,0.127347,0.286961,1.028406,1.508586,1.023149,2.249447,1.525613,High
3,0.592819,0.629285,0.681717,0.738796,0.571298,0.674071,0.607203,0.564238,0.741311,0.770940,...,-0.374100,0.115662,-0.335108,0.236700,0.952915,1.030876,1.031485,0.924837,0.925384,Medium
4,0.358679,0.361708,0.433146,0.796785,0.359648,0.407949,0.680859,0.685811,0.764193,0.772340,...,-0.301205,0.107165,-0.380125,-0.181585,1.018418,1.181818,1.114234,0.970498,0.914998,Medium
5,0.572026,0.453436,0.600712,0.626382,0.545993,0.455692,0.611951,0.493675,0.716722,0.821480,...,-0.224356,-0.283193,-0.402182,-0.082474,0.913846,1.120430,0.880450,1.887727,1.483402,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,0.801017,0.802525,0.344437,0.707799,0.837787,0.872268,0.861952,0.770241,0.588016,0.832817,...,-0.405125,-0.215237,-0.440957,-0.557297,1.025726,1.519597,1.030813,0.791378,0.536828,Low
997,0.685757,0.737480,0.669915,0.704997,0.736409,0.672730,0.759664,0.719471,0.757206,0.724271,...,-0.267478,-0.074270,-0.248173,0.031238,0.955907,1.002907,0.959140,0.784638,0.750396,High
998,0.593640,0.639694,0.531694,0.533188,0.638356,0.638990,0.664794,0.675209,0.453196,0.415541,...,-0.456215,-0.418937,-0.451399,-0.305660,0.905960,0.943122,0.952110,2.306574,2.328553,Low
999,0.564513,0.608847,0.643957,0.643431,0.610180,0.663946,0.642867,0.651872,0.685066,0.757385,...,-0.386168,-0.586159,-0.416468,-0.218402,1.023482,1.046446,0.973131,0.891722,0.829248,Low
